In [ ]:
#pip install google_play_scraper
#%pip install pandas

In [2]:
from google_play_scraper import Sort, reviews
import pandas as pd
import time, random

from datetime import datetime, timedelta
import os
import logging

In [ ]:
# esto es solo para una aplicacion y los primeros 200 comentarios

#app_id = "bo.com.yolopago"

# Extraer reseñas (puedes ajustar 'count' según lo que necesites)
#result, _ = reviews(
#    app_id,
#    lang='es',
#    country='bo',
#    sort=Sort.NEWEST, # puedes usar: Sort.NEWEST, Sort.HELPFUL, Sort.RATING
#    count=200         # número de reseñas a descargar en una corrida
#)

# Convertir a DataFrame
#df = pd.DataFrame(result)

# Guardar en CSV
#df.to_csv("comentarios_app.csv", index=False, encoding="utf-8-sig")

#print("Se guardaron", len(df), "comentarios en 'comentarios_app.csv'")


In [ ]:
#https://play.google.com/store/apps/details?id=bo.com.yolopago&pcampaignid=web_share
#https://play.google.com/store/apps/details?id=com.bancosol.altoke&pcampaignid=web_share
#https://play.google.com/store/apps/details?id=com.bcp.bo.wallet&pcampaignid=web_share
#https://play.google.com/store/apps/details?id=com.busa.wallet&pcampaignid=web_share
#https://play.google.com/store/apps/details?id=com.walletapp.mobile&pcampaignid=web_share

In [ ]:
#print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   reviewId              200 non-null    object        
 1   userName              200 non-null    object        
 2   userImage             200 non-null    object        
 3   content               200 non-null    object        
 4   score                 200 non-null    int64         
 5   thumbsUpCount         200 non-null    int64         
 6   reviewCreatedVersion  165 non-null    object        
 7   at                    200 non-null    datetime64[ns]
 8   replyContent          0 non-null      object        
 9   repliedAt             0 non-null      object        
 10  appVersion            165 non-null    object        
dtypes: datetime64[ns](1), int64(2), object(8)
memory usage: 17.3+ KB
None


In [ ]:
#df['content'].sample(15)

,content
22,muy útil!
115,"un desastre falla la billetera móvil,no puedo ..."
96,Muy buena aplicación
58,💪
99,"muy agradecida con el personal de atención, so..."
178,esa app esta a punto de dejar de ser compatibl...
79,pura burrera la nueva actualizacion solo es pa...
48,Muy buena🌟
107,interesante app
109,aurita mismo me están pidiendo q active mi ubi...


In [ ]:
all_reviews = []
token = None

app_id = ["com.bancosol.altoke", "com.bcp.bo.wallet", "bo.com.yolopago", "com.busa.wallet", "id=com.walletapp.mobile"]

for app in app_id:
    for _ in range(5):
        result, token = reviews(
            app,
            lang='es',
            country='bo',
            sort=Sort.NEWEST,
            count=200,
            continuation_token=token # esto te permite paginar
        )
        all_reviews.extend(result)

        if not token:  # cuando ya no hay más reseñas
            break
        time.sleep(random.uniform(5, 10))

    df = pd.DataFrame(all_reviews)
    df.to_csv(f"../data/restore/{app}.csv", index=False, encoding="utf-8-sig")
    print("✅ Total de reseñas descargadas:", len(df))


In [3]:
app_ids = [
    "com.bancosol.altoke", 
    "com.bcp.bo.wallet", 
    "bo.com.yolopago", 
    "com.busa.wallet", 
    "com.walletapp.mobile"]

In [4]:
"""
Script de Recolección Inicial - Apps Bolivia
Ejecutar UNA SOLA VEZ para obtener todo el historial de comentarios
"""

# Configuración de logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler(f'scraping_inicial_{datetime.now().strftime("%Y%m%d")}.log'),
        logging.StreamHandler()
    ]
)
logger = logging.getLogger(__name__)

class InitialScraper:
    def __init__(self):
        """Configuración específica para recolección inicial"""
        self.country = 'bo'  
        self.lang = 'es'     
        self.base_delay = (8, 15)  # Delays para recolección masiva
        self.max_retries = 3
        self.output_dir = "../data/initial_reviews"
        self.session_requests = 0
        
    def scrape_app_complete(self, app_id):
        """
        Recolecta TODOS los comentarios históricos de una app
        
        Args:
            app_id (str): ID de la aplicación (ej: 'com.whatsapp')
            
        Returns:
            list: todos los comentarios encontrados
        """
        logger.info(f"🚀 INICIANDO RECOLECCIÓN COMPLETA: {app_id}")
        
        all_reviews = []
        token = None
        page_count = 0
        consecutive_errors = 0
        
        start_time = datetime.now()
        
        while True:
            try:
                # Request a Google Play Store
                result, token = reviews(
                    app_id,
                    lang=self.lang,
                    country=self.country,
                    sort=Sort.NEWEST,  # Más recientes primero
                    count=200,         # Máximo por request
                    continuation_token=token
                )
                
                self.session_requests += 1
                
                # Verificar si hay resultados
                if not result:
                    logger.info(f"📄 Página {page_count}: Sin más comentarios")
                    break
                
                # Validar calidad de datos
                valid_reviews = self._validate_reviews(result, app_id)
                all_reviews.extend(valid_reviews)
                page_count += 1
                
                logger.info(f"📄 Página {page_count}: {len(result)} comentarios | "
                           f"Válidos: {len(valid_reviews)} | Total: {len(all_reviews)}")
                
                # Sin token = fin de datos
                if not token:
                    logger.info("✅ Token agotado - recolección completa")
                    break
                
                consecutive_errors = 0  # Reset contador de errores
                
                # Delay inteligente
                self._smart_delay(page_count)
                
            except Exception as e:
                consecutive_errors += 1
                logger.warning(f"⚠️ Error página {page_count}: {str(e)}")
                
                if consecutive_errors >= self.max_retries:
                    logger.error(f"❌ Máximo de errores alcanzado para {app_id}")
                    break
                
                # Delay por error (exponencial)
                error_delay = min(120, 20 * (2 ** consecutive_errors))
                logger.info(f"😴 Esperando {error_delay}s por error...")
                time.sleep(error_delay)
        
        duration = datetime.now() - start_time
        logger.info(f"🎯 COMPLETADO {app_id}: {len(all_reviews)} comentarios en {duration}")
        
        return all_reviews
    
    def _validate_reviews(self, reviews_batch, app_id):
        """Valida calidad de datos de comentarios"""
        valid_reviews = []
        issues = {'sin_contenido': 0, 'sin_fecha': 0, 'sin_puntuacion': 0}
        
        for review in reviews_batch:
            is_valid = True
            
            # Verificar contenido
            if not review.get('content') or len(review['content'].strip()) < 5:
                issues['sin_contenido'] += 1
                is_valid = False
            
            # Verificar fecha
            if not review.get('at') or not isinstance(review['at'], datetime):
                issues['sin_fecha'] += 1
                is_valid = False
            
            # Verificar puntuación
            if review.get('score') is None or not (1 <= review.get('score', 0) <= 5):
                issues['sin_puntuacion'] += 1
                is_valid = False
            
            if is_valid:
                # Agregar metadatos útiles
                review['app_id'] = app_id
                review['scraped_at'] = datetime.now()
                review['content_length'] = len(review['content'])
                valid_reviews.append(review)
        
        # Log de issues si existen
        if sum(issues.values()) > 0:
            logger.warning(f"⚠️ Issues encontrados: {issues}")
        
        return valid_reviews
    
    def _smart_delay(self, page_count):
        """Sistema de delays inteligente para evitar bloqueos"""
        base_min, base_max = self.base_delay
        
        # Incremento gradual por volumen de requests
        volume_multiplier = 1 + (self.session_requests // 100) * 0.3
        
        # Delays especiales en puntos críticos
        if page_count % 25 == 0:  # Cada 25 páginas (5000 comentarios)
            special_delay = random.uniform(45, 90)
            logger.info(f"🛑 PAUSA ESPECIAL (página {page_count}): {special_delay:.1f}s")
            time.sleep(special_delay)
            return
        
        # Delay normal con variación
        normal_delay = random.uniform(
            base_min * volume_multiplier,
            base_max * volume_multiplier
        )
        
        logger.debug(f"⏳ Delay: {normal_delay:.1f}s")
        time.sleep(normal_delay)
    
    def save_complete_data(self, reviews, app_id):
        """
        Guarda datos completos con metadata rica
        """
        if not reviews:
            logger.warning(f"❌ Sin datos para guardar: {app_id}")
            return None
        
        # Crear directorio
        os.makedirs(self.output_dir, exist_ok=True)
        
        # DataFrame con datos completos
        df = pd.DataFrame(reviews)
        
        # Estadísticas pre-guardado
        stats = self._generate_stats(df, app_id)
        
        # Archivo principal con timestamp
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        main_file = f"{app_id}_complete_{timestamp}.csv"
        main_path = os.path.join(self.output_dir, main_file)
        
        # Guardar CSV principal
        df.to_csv(main_path, index=False, encoding="utf-8-sig")
        
        # Guardar archivo de estadísticas
        stats_file = f"{app_id}_stats_{timestamp}.txt"
        stats_path = os.path.join(self.output_dir, stats_file)
        
        with open(stats_path, 'w', encoding='utf-8') as f:
            f.write(stats)
        
        logger.info("💾 GUARDADO EXITOSO:")
        logger.info(f"   📁 Datos: {main_file}")
        logger.info(f"   📊 Stats: {stats_file}")
        
        return main_path
    
    def _generate_stats(self, df, app_id):
        """Genera estadísticas detalladas"""
        stats_text = f"""
ESTADÍSTICAS DE RECOLECCIÓN INICIAL
=====================================
App ID: {app_id}
Fecha de scraping: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}

VOLUMEN DE DATOS:
• Total comentarios: {len(df):,}
• Rango temporal: {df['at'].min().date()} → {df['at'].max().date()}
• Días cubiertos: {(df['at'].max() - df['at'].min()).days}

DISTRIBUCIÓN DE PUNTUACIONES:
"""
        
        for score in range(1, 6):
            count = len(df[df['score'] == score])
            percentage = (count / len(df)) * 100
            stats_text += f"• {score} estrellas: {count:,} ({percentage:.1f}%)\n"
        
        stats_text += f"""
MÉTRICAS DE CONTENIDO:
• Promedio caracteres: {df['content_length'].mean():.0f}
• Comentario más largo: {df['content_length'].max()} caracteres
• Comentario más corto: {df['content_length'].min()} caracteres

ACTIVIDAD TEMPORAL:
• Comentarios último mes: {len(df[df['at'] > datetime.now() - pd.DateOffset(months=1)]):,}
• Comentarios últimos 7 días: {len(df[df['at'] > datetime.now() - pd.DateOffset(days=7)]):,}
• Puntuación promedio: {df['score'].mean():.2f}/5.0
"""
        
        return stats_text

def main():
    """Función principal para ejecutar recolección inicial"""
    
    # 🔧 CONFIGURAR AQUÍ TUS APP IDs
    # app_ids esta definida una celda arriba
    
    if not app_ids:
        logger.warning("⚠️ CONFIGURAR app_ids en la variable antes de ejecutar")
        print("\n📋 Para usar este script:")
        print("1. Edita la variable 'app_ids' en main()")
        print("2. Agrega los IDs de las apps bolivianas a analizar")
        print("3. Ejecuta: python initial_scraper.py")
        return
    
    scraper = InitialScraper()
    results = {}
    
    logger.info(f"🎯 INICIANDO RECOLECCIÓN INICIAL DE {len(app_ids)} APPS")
    logger.info("📍 País: Bolivia | Idioma: Español")
    
    for i, app_id in enumerate(app_ids, 1):
        logger.info(f"\n{'='*60}")
        logger.info(f"📱 PROCESANDO APP {i}/{len(app_ids)}: {app_id}")
        logger.info(f"{'='*60}")
        
        try:
            # Recolectar todos los comentarios
            reviews = scraper.scrape_app_complete(app_id)
            
            if reviews:
                # Guardar datos
                saved_path = scraper.save_complete_data(reviews, app_id)
                results[app_id] = {
                    'status': 'success',
                    'count': len(reviews),
                    'file': saved_path
                }
                logger.info(f"✅ {app_id}: {len(reviews):,} comentarios guardados")
            else:
                results[app_id] = {'status': 'no_data', 'count': 0, 'file': None}
                logger.warning(f"⚠️ {app_id}: Sin comentarios encontrados")
        
        except Exception as e:
            logger.error(f"❌ ERROR CRÍTICO en {app_id}: {str(e)}")
            results[app_id] = {'status': 'error', 'count': 0, 'error': str(e)}
        
        # Pausa larga entre apps
        if i < len(app_ids):
            inter_app_delay = random.uniform(60, 120)  # 1-2 minutos entre apps
            logger.info(f"🔄 Pausa entre apps: {inter_app_delay:.1f}s")
            time.sleep(inter_app_delay)
    
    # Resumen final
    _print_final_summary(results)

def _print_final_summary(results):
    """Imprime resumen final de la recolección"""
    print(f"\n{'='*80}")
    print("🎉 RECOLECCIÓN INICIAL COMPLETADA")
    print(f"{'='*80}")
    
    total_reviews = sum(r.get('count', 0) for r in results.values())
    successful = sum(1 for r in results.values() if r.get('status') == 'success')
    
    print("📊 RESUMEN GENERAL:")
    print(f"   • Apps procesadas: {len(results)}")
    print(f"   • Apps exitosas: {successful}")
    print(f"   • Total comentarios: {total_reviews:,}")
    
    print("\n📱 DETALLE POR APP:")
    for app_id, result in results.items():
        status_icon = "✅" if result['status'] == 'success' else "❌"
        print(f"   {status_icon} {app_id}: {result.get('count', 0):,} comentarios")
    
    print("\n📁 Archivos guardados en: ../data/initial_reviews/")
    print(f"📝 Log completo en: scraping_inicial_{datetime.now().strftime('%Y%m%d')}.log")

if __name__ == "__main__":
    main()

--- Logging error ---
Traceback (most recent call last):
  File "C:\Users\OrdonezM\AppData\Local\Programs\Python\Python311\Lib\logging\__init__.py", line 1113, in emit
    stream.write(msg + self.terminator)
  File "C:\Users\OrdonezM\AppData\Local\Programs\Python\Python311\Lib\encodings\cp1252.py", line 19, in encode
    return codecs.charmap_encode(input,self.errors,encoding_table)[0]
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
UnicodeEncodeError: 'charmap' codec can't encode character '\U0001f3af' in position 33: character maps to <undefined>
Call stack:
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "f:\modulo7\proyecto-playstore\.venv\Lib\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "f:\modulo7\proyecto-playstore\.venv\Lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start()
  File "f:\modulo7\proyecto-pl


🎉 RECOLECCIÓN INICIAL COMPLETADA
📊 RESUMEN GENERAL:
   • Apps procesadas: 5
   • Apps exitosas: 5
   • Total comentarios: 16,103

📱 DETALLE POR APP:
   ✅ com.bancosol.altoke: 1,552 comentarios
   ✅ com.bcp.bo.wallet: 10,698 comentarios
   ✅ bo.com.yolopago: 2,725 comentarios
   ✅ com.busa.wallet: 1,076 comentarios
   ✅ com.walletapp.mobile: 52 comentarios

📁 Archivos guardados en: ../data/initial_reviews/
📝 Log completo en: scraping_inicial_20250928.log


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   reviewId              3000 non-null   object        
 1   userName              3000 non-null   object        
 2   userImage             3000 non-null   object        
 3   content               3000 non-null   object        
 4   score                 3000 non-null   int64         
 5   thumbsUpCount         3000 non-null   int64         
 6   reviewCreatedVersion  2447 non-null   object        
 7   at                    3000 non-null   datetime64[ns]
 8   replyContent          772 non-null    object        
 9   repliedAt             772 non-null    datetime64[ns]
 10  appVersion            2447 non-null   object        
dtypes: datetime64[ns](2), int64(2), object(7)
memory usage: 257.9+ KB


In [8]:
df2['content'].sample(12)

305     ya van 3 veces que hago transferencia en tiend...
375                                                 super
1843    La verdad no se porque pide actualizar y me pu...
235     una basura me robó 6 bs y se cuelga al momento...
2051                                   Pele la aplicación
1761    Que Pasa Con Yape Esta Caido Me Estan Perjudic...
1222    Yape se cuelga 3 veces a la semana en Bolivia....
2245    Es buena la aplicación me ha servido de mucha ...
355     es burocrático te pide carnet de identidad al ...
1500    Que pasó con las recargas de paquetes? Ya no s...
2275    La aplicación funciona realmente pésimo .... C...
827     si yo igual creado mi cuenta ahora pago mis fa...
Name: content, dtype: object

In [9]:
df2.shape

(3000, 11)

In [11]:
df2.to_csv("../data/restore/comentarios_app.csv", index=False, encoding="utf-8-sig")
print("✅ Total de reseñas descargadas:", len(df2))

✅ Total de reseñas descargadas: 3000


In [ ]:
"""
Script de Recolección Diaria - Apps Bolivia
Ejecutar DIARIAMENTE para obtener comentarios del día anterior
Optimizado para volúmenes pequeños (~1000 comentarios por app)
"""

import pandas as pd
import time
import random
from google_play_scraper import reviews, Sort
from datetime import datetime, timedelta
import os
import logging
import json

# Configuración de logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler(f'scraping_diario_{datetime.now().strftime("%Y%m%d")}.log'),
        logging.StreamHandler()
    ]
)
logger = logging.getLogger(__name__)

class DailyScraper:
    def __init__(self):
        """Configuración específica para recolección diaria"""
        self.country = 'bo'  # Bolivia fijo
        self.lang = 'es'     # Español fijo
        self.base_delay = (3, 6)  # Delays más rápidos para volúmenes pequeños
        self.max_pages = 15      # Límite de páginas para búsqueda diaria
        self.output_dir = "../data/daily_reviews"
        self.config_file = "../config/daily_scraper_config.json"
        
    def get_yesterday_reviews(self, app_id, target_date=None):
        """
        Busca comentarios de una fecha específica (por defecto ayer)
        
        Args:
            app_id (str): ID de la aplicación
            target_date (date): fecha objetivo, por defecto ayer
            
        Returns:
            list: comentarios de la fecha especificada
        """
        if target_date is None:
            target_date = (datetime.now() - timedelta(days=1)).date()
        
        logger.info(f"📅 Buscando comentarios del {target_date} para {app_id}")
        
        daily_reviews = []
        token = None
        pages_checked = 0
        found_target_date = False
        older_date_found = False
        
        while pages_checked < self.max_pages and not older_date_found:
            try:
                # Request más pequeño y rápido
                result, token = reviews(
                    app_id,
                    lang=self.lang,
                    country=self.country,
                    sort=Sort.NEWEST,  # Más recientes primero (crítico para búsqueda diaria)
                    count=200,
                    continuation_token=token
                )
                
                if not result:
                    logger.info(f"📄 Sin más comentarios después de {pages_checked} páginas")
                    break
                
                pages_checked += 1
                page_daily_count = 0
                
                for review in result:
                    if not review.get('at'):
                        continue
                    
                    review_date = review['at'].date()
                    
                    if review_date == target_date:
                        # Validar y agregar comentario del día objetivo
                        if self._is_valid_daily_review(review):
                            review['app_id'] = app_id
                            review['scraped_at'] = datetime.now()
                            daily_reviews.append(review)
                            page_daily_count += 1
                            found_target_date = True
                    
                    elif review_date < target_date:
                        # Hemos pasado la fecha objetivo
                        older_date_found = True
                        break
                
                logger.info(f"📄 Página {pages_checked}: {page_daily_count} comentarios del {target_date}")
                
                if not token or older_date_found:
                    break
                
                # Delay corto para recolección diaria
                time.sleep(random.uniform(2, 4))
                
            except Exception as e:
                logger.warning(f"⚠️ Error en página {pages_checked}: {str(e)}")
                time.sleep(10)  # Pausa corta y continuar
                break
        
        if found_target_date:
            logger.info(f"✅ {app_id}: {len(daily_reviews)} comentarios encontrados para {target_date}")
        else:
            logger.warning(f"⚠️ {app_id}: Sin comentarios para {target_date}")
        
        return daily_reviews
    
    def _is_valid_daily_review(self, review):
        """Validación rápida para comentarios diarios"""
        return (
            review.get('content') and 
            len(review['content'].strip()) >= 3 and
            review.get('score') is not None and
            1 <= review.get('score', 0) <= 5
        )
    
    def save_daily_data(self, reviews, app_id, target_date):
        """
        Guarda comentarios diarios con formato estandarizado
        """
        if not reviews:
            logger.info(f"📝 {app_id}: Sin comentarios para {target_date}")
            return None
        
        # Crear directorio con estructura por año/mes
        year_month = target_date.strftime("%Y/%m")
        full_dir = os.path.join(self.output_dir, year_month)
        os.makedirs(full_dir, exist_ok=True)
        
        # Nombre de archivo estandarizado: app_YYYYMMDD.csv
        filename = f"{app_id}_{target_date.strftime('%Y%m%d')}.csv"
        filepath = os.path.join(full_dir, filename)
        
        # DataFrame con datos del día
        df = pd.DataFrame(reviews)
        
        # Agregar columnas útiles para análisis
        df['review_date'] = target_date
        df['weekday'] = target_date.strftime('%A')
        df['content_words'] = df['content'].str.split().str.len()
        
        # Guardar
        df.to_csv(filepath, index=False, encoding="utf-8-sig")
        
        # Estadísticas rápidas
        avg_score = df['score'].mean()
        logger.info(f"💾 {app_id}: {len(df)} comentarios | Promedio: {avg_score:.2f}⭐")
        logger.info(f"   📁 Guardado en: {filename}")
        
        return filepath
    
    def load_app_config(self):
        """Carga configuración de apps desde archivo JSON"""
        try:
            if os.path.exists(self.config_file):
                with open(self.config_file, 'r', encoding='utf-8') as f:
                    config = json.load(f)
                    return config.get('app_ids', [])
            else:
                # Crear archivo de configuración ejemplo
                self._create_example_config()
                return []
        except Exception as e:
            logger.warning(f"⚠️ Error cargando configuración: {str(e)}")
            return []
    
    def _create_example_config(self):
        """Crea archivo de configuración de ejemplo"""
        os.makedirs(os.path.dirname(self.config_file), exist_ok=True)
        
        example_config = {
            "app_ids": [
                "com.bancosol.altoke", 
                "com.bcp.bo.wallet", 
                "bo.com.yolopago", 
                "com.busa.wallet", 
                "com.walletapp.mobile"
            ],
            "notes": "Agregar aquí los IDs de las apps  a monitorear diariamente"
        }
        
        with open(self.config_file, 'w', encoding='utf-8') as f:
            json.dump(example_config, f, indent=2, ensure_ascii=False)
        
        logger.info(f"📝 Archivo de configuración creado: {self.config_file}")
    
    def run_daily_collection(self, target_date=None, app_ids=None):
        """
        Ejecuta recolección diaria completa
        
        Args:
            target_date: fecha objetivo (por defecto ayer)
            app_ids: lista de apps (por defecto desde config)
        """
        if target_date is None:
            target_date = (datetime.now() - timedelta(days=1)).date()
        
        if app_ids is None:
            app_ids = self.load_app_config()
        
        if not app_ids:
            logger.error("❌ Sin apps configuradas para monitoreo diario")
            logger.info("💡 Editar archivo: ../config/daily_scraper_config.json")
            return {}
        
        logger.info(f"🎯 RECOLECCIÓN DIARIA: {target_date}")
        logger.info(f"📱 Apps a procesar: {len(app_ids)}")
        
        results = {}
        start_time = datetime.now()
        
        for i, app_id in enumerate(app_ids, 1):
            logger.info(f"\n--- Procesando {i}/{len(app_ids)}: {app_id} ---")
            
            try:
                # Buscar comentarios del día
                daily_reviews = self.get_yesterday_reviews(app_id, target_date)
                
                # Guardar si hay datos
                if daily_reviews:
                    saved_path = self.save_daily_data(daily_reviews, app_id, target_date)
                    results[app_id] = {
                        'status': 'success',
                        'count': len(daily_reviews),
                        'file': saved_path,
                        'avg_score': sum(r['score'] for r in daily_reviews) / len(daily_reviews)
                    }
                else:
                    results[app_id] = {
                        'status': 'no_data',
                        'count': 0,
                        'file': None,
                        'avg_score': None
                    }
                
            except Exception as e:
                logger.error(f"❌ Error procesando {app_id}: {str(e)}")
                results[app_id] = {
                    'status': 'error',
                    'count': 0,
                    'error': str(e)
                }
            
            # Pausa corta entre apps (5-10 segundos)
            if i < len(app_ids):
                time.sleep(random.uniform(5, 10))
        
        # Resumen final
        duration = datetime.now() - start_time
        self._print_daily_summary(results, target_date, duration)
        
        return results
    
    def _print_daily_summary(self, results, target_date, duration):
        """Imprime resumen de recolección diaria"""
        print(f"\n{'='*60}")
        print(f"📊 RESUMEN DIARIO - {target_date}")
        print(f"{'='*60}")
        
        total_reviews = sum(r.get('count', 0) for r in results.values())
        successful = sum(1 for r in results.values() if r.get('status') == 'success')
        no_data = sum(1 for r in results.values() if r.get('status') == 'no_data')
        
        print(f"⏱️ Tiempo total: {duration}")
        print(f"📱 Apps procesadas: {len(results)}")
        print(f"✅ Con comentarios: {successful}")
        print(f"📝 Sin comentarios: {no_data}")
        print(f"📊 Total comentarios: {total_reviews}")
        
        if successful > 0:
            print("\n📈 DETALLE POR APP:")
            for app_id, result in results.items():
                if result['status'] == 'success':
                    avg = result.get('avg_score', 0)
                    print(f"   📱 {app_id}: {result['count']} comentarios ({avg:.1f}⭐)")
        
        print(f"\n📁 Datos guardados en: {self.output_dir}")

def main():
    """Función principal - puede ejecutarse desde cron job"""
    scraper = DailyScraper()
    
    # Para ejecución manual con fecha específica
    import sys
    if len(sys.argv) > 1:
        try:
            target_date = datetime.strptime(sys.argv[1], '%Y-%m-%d').date()
            logger.info(f"📅 Fecha especificada: {target_date}")
        except ValueError:
            logger.error("❌ Formato de fecha inválido. Usar: YYYY-MM-DD")
            return
    else:
        target_date = None  # Por defecto ayer
    
    # Ejecutar recolección
    results = scraper.run_daily_collection(target_date)
    
    # Para integración con sistemas de monitoreo
    success_count = sum(1 for r in results.values() if r.get('status') == 'success')
    total_reviews = sum(r.get('count', 0) for r in results.values())
    
    logger.info(f"🎯 Finalizado: {success_count} apps exitosas, {total_reviews} comentarios")

if __name__ == "__main__":
    main()